In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imblearn.over_sampling as oversampling
import imblearn.under_sampling as undersampling
import sklearn.model_selection as selection
import sklearn.metrics as metrics
import sklearn.preprocessing as preprocessing

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, max_error

# Задание

<b>Выполните следующие задания:</b>
- решите задачи регрессии и классификации на ваших данных используя полносвязные нейронные сети; соберите их используя API Keras фреймворка TensorFlow; оцените качество полученных моделей с помощью метрик; 
- реализуйте многослойный персептрон, с помощью которого можно решать задачи регрессии и классификации; предусмотрите возможность использовать такие функции активации, как sigmoid, tanh и relu; также предусмотрите возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой; реализуйте обучение персептрона методом обратного распространения ошибки; самостоятельно найдите производные функций sigmoid, tanh и relu; реализуйте классический градиентный спуск с возможностью указания шага.

## Предобработка (регрессия)

In [4]:
reg = pd.read_csv('../datasets/raw/cars.csv')
reg['engine_capacity'].fillna(0, inplace=True)
reg.drop(columns=['engine_fuel', 'model_name', 'color', 'engine_type', 'body_type', 'drivetrain', 'location_region'], inplace=True)
reg = pd.get_dummies(reg, columns=['manufacturer_name', 'transmission', 'state'])

In [5]:
y_reg = reg['price_usd']
X_reg = reg.drop(columns=['price_usd'])

In [6]:
X_reg

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,is_exchangeable,number_of_photos,up_counter,feature_0,feature_1,...,manufacturer_name_ВАЗ,manufacturer_name_ГАЗ,manufacturer_name_ЗАЗ,manufacturer_name_Москвич,manufacturer_name_УАЗ,transmission_automatic,transmission_mechanical,state_emergency,state_new,state_owned
0,190000,2010,False,2.5,False,False,9,13,False,True,...,0,0,0,0,0,1,0,0,0,1
1,290000,2002,False,3.0,False,True,12,54,False,True,...,0,0,0,0,0,1,0,0,0,1
2,402000,2001,False,2.5,False,True,4,72,False,True,...,0,0,0,0,0,1,0,0,0,1
3,10000,1999,False,3.0,False,True,9,42,True,False,...,0,0,0,0,0,0,1,0,0,1
4,280000,2001,False,2.5,False,True,14,7,False,True,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,False,3.5,False,True,5,85,False,True,...,0,0,0,0,0,1,0,0,0,1
38527,321000,2004,False,2.2,False,True,4,20,False,True,...,0,0,0,0,0,0,1,0,0,1
38528,777957,2000,False,3.5,False,False,3,63,False,True,...,0,0,0,0,0,1,0,0,0,1
38529,20000,2001,False,2.0,False,True,7,156,False,True,...,0,0,0,0,0,0,1,0,0,1


## Предобработка (классификация)

In [7]:
cl = pd.read_csv('../datasets/raw/diabetes.csv')
cl = cl.sample(frac=1)

In [8]:
from sklearn.utils import resample

data_0 = cl[cl['Diabetes_012'] == 0]
data_2 = cl[cl['Diabetes_012'] == 2]

data_0 = resample(data_0, replace=False, n_samples=5000)
data_2 = resample(data_2, replace=True, n_samples=5000)

cl = pd.concat([data_0, data_2])

y_cl = cl['Diabetes_012']
X_cl = cl.drop(columns=['Diabetes_012'])

X_cl

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
113488,1.0,1.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,10.0,6.0,7.0
241077,0.0,0.0,1.0,29.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,4.0,1.0
188089,1.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,7.0,5.0,8.0
70283,0.0,1.0,0.0,23.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,3.0,0.0,2.0,0.0,0.0,10.0,6.0,4.0
204838,0.0,1.0,1.0,23.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,3.0,5.0,2.0,0.0,0.0,6.0,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106818,1.0,1.0,1.0,31.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,13.0,3.0,4.0
82575,1.0,1.0,1.0,45.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,3.0,3.0,15.0,1.0,0.0,11.0,5.0,2.0
132861,1.0,1.0,1.0,27.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,5.0,10.0,25.0,1.0,1.0,10.0,2.0,2.0
203542,1.0,1.0,1.0,23.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,4.0,30.0,28.0,1.0,1.0,6.0,5.0,7.0


## Выборки

In [32]:
from sklearn.model_selection import train_test_split

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2)

# X_cl_train, X_cl_test, y_cl_train, y_cl_test = train_test_split(X_cl, y_cl, stratify=y_cl, test_size=0.2)

X_cl_train, X_cl_test, y_cl_train, y_cl_test = train_test_split(X_cl, y_cl, test_size=0.2)

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report

# Нейронная сеть (регрессия)

In [11]:
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(X_reg_train.shape[1],)),

        tf.keras.layers.Dense(32, activation="linear"),

        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [12]:
model_regression.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                5120      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 7,745
Trainable params: 7,745
Non-trainabl

In [13]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [14]:
model_regression.fit(X_reg_train.to_numpy().astype(np.float32), y_reg_train.to_numpy().astype(np.float32), epochs=50)

Epoch 1/50
964/964 [==============================] - 2s 2ms/step - loss: 52580204.0000
Epoch 2/50
964/964 [==============================] - 2s 2ms/step - loss: 33305464.0000
Epoch 3/50
964/964 [==============================] - 2s 2ms/step - loss: 32397188.0000
Epoch 4/50
964/964 [==============================] - 2s 2ms/step - loss: 31424072.0000
Epoch 5/50
964/964 [==============================] - 2s 2ms/step - loss: 30536102.0000
Epoch 6/50
964/964 [==============================] - 2s 2ms/step - loss: 29722162.0000
Epoch 7/50
964/964 [==============================] - 2s 2ms/step - loss: 29319494.0000
Epoch 8/50
964/964 [==============================] - 1s 2ms/step - loss: 28700390.0000
Epoch 9/50
964/964 [==============================] - 1s 1ms/step - loss: 27389640.0000
Epoch 10/50
964/964 [==============================] - 1s 1ms/step - loss: 26585330.0000
Epoch 11/50
964/964 [==============================] - 1s 1ms/step - loss: 24848310.0000
Epoch 12/50
964/964 [=========

In [20]:
print(mean_absolute_error(y_reg_test, model_regression.predict(X_reg_test.to_numpy().astype(np.float32))))
print(mean_squared_error(y_reg_test, model_regression.predict(X_reg_test.to_numpy().astype(np.float32))))
print(r2_score(y_reg_test, model_regression.predict(X_reg_test.to_numpy().astype(np.float32))))

241/241 [==============================] - 0s 1ms/step
3052.920216520138
241/241 [==============================] - 0s 2ms/step
21212199.574690133
241/241 [==============================] - 0s 1ms/step
0.4705949152564147


# Нейронная сеть (классификация)

In [16]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(X_cl_train.shape[1],)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")

model_classification_1.fit(X_cl_train, y_cl_train, epochs=25)

Epoch 1/25
250/250 [==============================] - 1s 2ms/step - loss: 0.9060
Epoch 2/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8567
Epoch 3/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8460
Epoch 4/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8353
Epoch 5/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8322
Epoch 6/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8295
Epoch 7/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8283
Epoch 8/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8280
Epoch 9/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8281
Epoch 10/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8230
Epoch 11/25
250/250 [==============================] - 0s 2ms/step - loss: 0.8239
Epoch 12/25
250/250 [==============================] - 1s 2ms/step - loss: 0.8238
Epoch 13/25
250/250 [====

In [17]:
model_classification_1.predict(X_cl_test)[:5]

63/63 [==============================] - 0s 1ms/step


array([[0.16919261],
       [0.9999857 ],
       [0.6070916 ],
       [0.99999917],
       [0.42494455]], dtype=float32)

In [18]:
y_pred = np.around(model_classification_1.predict(X_cl_test, verbose=None))

In [19]:
print(classification_report(y_cl_test, y_pred))
print(confusion_matrix(y_cl_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.51      0.65      1000
         1.0       0.00      0.00      0.00         0
         2.0       0.00      0.00      0.00      1000

    accuracy                           0.26      2000
   macro avg       0.30      0.17      0.22      2000
weighted avg       0.45      0.26      0.33      2000

[[513 487   0]
 [  0   0   0]
 [ 60 940   0]]


C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

# Реализация

In [77]:
def iterate_batches(inputs, outputs, batchsize): # получает х, у и и размерноcть батчсайза и делит на батчи  размерность батчсайза
    l = len(inputs)
    for ndx in range(0, l, batchsize):
        yield inputs[ndx:min(ndx + batchsize, l)], outputs[ndx:min(ndx + batchsize, l)]

class DenseLayer:
    def __init__(self, n_units, input_size=None, activation=None): # нейроны, размерность входных данных
        
        self.n_units = n_units                                    
        self.input_size = input_size                               
        self.W = None                                                                                      
        self.A = None                            
        self.fn, self.df = self.select_activation_fn(activation)

    def select_activation_fn(self, activation): 
        if activation == 'relu':
            fn = lambda x: np.maximum(0, x)
            df = lambda x: np.where(x < 0, 0.0, 1.0)
        elif activation == 'sigmoid':
            fn = lambda x: 1 / (1 + np.exp(-x))
            df = lambda x: fn(x) * fn(1 - x)
        elif activation == 'tanh':
            fn = lambda x: (np.exp(x) - np.exp(-1)) / (np.exp(x) + np.exp(-x))
            df = lambda x: 1 / np.cos(x) ** 2
        elif activation == 'linear':
            fn = lambda x: x
            df = lambda x: 1.0
        return fn, df

    def init_weights(self): # normal distribution
        self.W = np.random.randn(self.n_units, self.input_size + 1)
    
    def get_activation_value(self, X):
        m_examples = X.shape[0] # сохранили количество объектов
        X_new = np.hstack([np.ones((m_examples, 1)), X]) # добавили столбец для смещения
        Z = np.dot(X_new, self.W.T) 
        A = self.fn(Z)
        self.A = A
        return A
    
    def backprop(self, delta, a):
        da = self.df(a)    
        return np.dot(delta, self.W)[:, 1:] * da

class SequentialModel:
    def __init__(self, layers):    
        input_size = layers[0].n_units # количество нейронов 1 слоя      
        layers[0].init_weights()       
        for layer in layers[1:]:       
            layer.input_size = input_size
            input_size = layer.n_units
            layer.init_weights()
        self.layers = layers
        self.lr = None

    def forward(self, X): # для каждого слоя считаем значение функции активации           
        out = self.layers[0].get_activation_value(X)
        for layer in self.layers[1:]:
            out = layer.get_activation_value(out) # возвращаем значение функции активации на последнем слое
        return out
    
    def add_dim(self, x):
        return np.hstack([np.ones((x.shape[0], 1)), x])

    def backward(self, X, y_pred, y_true): # обратное распространение 
        n_layers = len(self.layers)
        delta = y_pred - y_true
        a = y_pred
        dWs = {}
        for i in range(-1, -len(self.layers), -1): # используем mse 
            a = self.layers[i - 1].A       
            dWs[i] = np.dot(delta.T, self.add_dim(a))
            delta = self.layers[i].backprop(delta, a)

        dWs[-n_layers] = np.dot(delta.T, self.add_dim(X))

        for k, dW in dWs.items():
#             print(self.lr*dW)
            self.layers[k].W -= self.lr * dW.astype(float)
            
    def fit(self, X, y, epochs, batchsize=32, lr=0.01):
        
        self.lr = lr

        for i in range(epochs):

            for batch in iterate_batches(X, y, batchsize=batchsize):

                X_batch = batch[0].to_numpy()
                y_batch = batch[1].to_numpy()

            y_batch = y_batch.reshape(-1, 1)
            y_pred = model.forward(X_batch)
            model.backward(X_batch, y_pred, y_batch)


In [78]:
model = SequentialModel([
DenseLayer(6, activation='sigmoid', input_size=X_cl.shape[1]),
DenseLayer(4, activation='tanh'),
DenseLayer(3, activation='relu'),
DenseLayer(1, activation='sigmoid')
])

model.fit(X_cl_train, y_cl_train, epochs = 5, batchsize=32)
pred = model.forward(X_cl_test)
pred = np.where(pred > 0.5, 0, 2)
# print(X_cl_train.shape, X_cl_test.shape, y_cl_test.shape, pred.shape)
print(classification_report(y_cl_test, pred))


model = SequentialModel([
DenseLayer(6, activation='relu', input_size=X_reg.shape[1]),
DenseLayer(4, activation='linear'),
DenseLayer(3, activation='relu'),
DenseLayer(1, activation='linear')
])

model.fit(X_reg_train, y_reg_train, epochs = 25, batchsize=32, )
pred = model.forward(X_reg_test)
pred = np.where(pred > 0.5, 1, 0)
print(f'MAE: {mean_absolute_error(y_reg_test, pred)}')
print(f'MSE: {mean_squared_error(y_reg_test, pred)}')


C:\Users\Егор\AppData\Local\Temp\ipykernel_9476\203172270.py:20: RuntimeWarning: overflow encountered in exp
  fn = lambda x: 1 / (1 + np.exp(-x))
C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Егор\Desktop\programming\ML\lab1\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being s

              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67      1000
         2.0       0.00      0.00      0.00      1000

    accuracy                           0.50      2000
   macro avg       0.25      0.50      0.33      2000
weighted avg       0.25      0.50      0.33      2000



C:\Users\Егор\AppData\Local\Temp\ipykernel_9476\203172270.py:17: RuntimeWarning: invalid value encountered in maximum
  fn = lambda x: np.maximum(0, x)
C:\Users\Егор\AppData\Local\Temp\ipykernel_9476\203172270.py:18: RuntimeWarning: invalid value encountered in less
  df = lambda x: np.where(x < 0, 0.0, 1.0)


MAE: 6564.272519787207
MSE: 82283133.21285816


C:\Users\Егор\AppData\Local\Temp\ipykernel_9476\168539864.py:24: RuntimeWarning: invalid value encountered in greater
  pred = np.where(pred > 0.5, 1, 0)
